# RAG: Function Calling

### 진행 과정

이 가이드북은 다음 워크플로우를 안내한다:

  - **검색 유틸리티**: 답변을 위해 arXiv에 접근하는 두 함수를 생성한다.
  - **에이전트 구성**: 함수 필요성을 평가하고, 필요한 경우 해당 함수를 호출하여 결과를 에이전트에게 다시 제시하는 에이전트 행동을 구축한다.
  - **arXiv 대화**: 이 모든 것을 실제 대화에 통합하여 실행한다.

In [2]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

### 1\. 설정 및 라이브러리 설치

In [ ]:
%pip install -q scipy tenacity tiktoken arxiv PyPDF2 tqdm

In [3]:
# 필요한 라이브러리들을 가져온다.
import arxiv
import ast 
import concurrent.futures 
import json
import os

import pandas as pd
import tiktoken 
from csv import writer

from openai import OpenAI
from PyPDF2 import PdfReader
from scipy import spatial 
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm 


# 사용할 GPT 모델과 임베딩 모델을 지정한다.

EMBEDDING_MODEL= "text-embedding-3-small"
MODEL = "gpt-4.1-mini"

client = OpenAI()

### 2\. 검색 유틸리티 (Search Utilities)

먼저 두 함수를 뒷받침할 몇 가지 유틸리티를 설정한다.

다운로드된 논문은 특정 디렉토리(여기서는 `./data/papers`)에 저장된다. `arxiv_library.csv` 파일을 생성하여 다운로드된 논문의 임베딩과 세부 정보를 저장하고, 나중에 `summarize_text`를 사용하여 검색할 수 있도록 한다.

  * **임베딩(Embedding)이란?** 텍스트(단어, 문장 등)를 컴퓨터가 이해할 수 있는 숫자의 벡터(vector)로 변환하는 기술이다. 이 벡터들은 의미적으로 유사한 텍스트일수록 서로 가까운 위치에 있게 되므로, 텍스트 간의 관련성을 계산하는 데 사용된다.

<!-- end list -->

In [4]:
# 다운로드한 논문을 저장할 디렉토리를 지정한다.
directory = '../dataset/papers'

# 디렉토리가 이미 존재하는지 확인한다.
if not os.path.exists(directory):
    # 디렉토리가 없으면, 필요한 모든 중간 디렉토리와 함께 생성한다.
    os.makedirs(directory)
    print(f"디렉토리 '{directory}'가 성공적으로 생성되었습니다.")
else:
    # 디렉토리가 이미 존재하면, 메시지를 출력한다.
    print(f"디렉토리 '{directory}'가 이미 존재합니다.")

디렉토리 './data/papers'가 이미 존재합니다.


In [5]:
# 다운로드된 논문을 저장할 디렉토리 경로를 설정한다.
data_dir = os.path.join(os.curdir, "data", "papers")
# 다운로드된 논문들의 메타데이터(제목, 파일 경로, 임베딩)를 저장할 CSV 파일 경로
paper_dir_filepath = "../dataset/papers/arxiv_library.csv"

# 비어있는 DataFrame을 생성하여 CSV 파일을 초기화한다.
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath, index=False)

In [6]:
# 요청 실패 시 지수적으로 증가하는 대기 시간(최소 1초, 최대 40초)으로 최대 3번 재시도한다.
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    """지정된 텍스트에 대한 임베딩을 요청한다."""
    
    response = client.embeddings.create(input=text, model=EMBEDDING_MODEL)
    return response

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def get_articles(query, library=paper_dir_filepath, top_k=10):
    """
    이 함수는 사용자의 쿼리를 기반으로 관련성이 높은 상위 k개의 논문을 가져온다.
    또한 파일을 다운로드하고, 나중에 read_article_and_summarize 함수가 검색할 수 있도록
    arxiv_library.csv에 저장한다.
    """
    client = arxiv.Client()
    # 쿼리로 arXiv에서 논문을 검색한다.
    search = arxiv.Search(
        query = query,
        max_results = top_k
    )
    result_list = []
    for result in client.results(search):
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})
        # 첫 번째 URL을 논문 URL로 사용한다.
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # 논문 제목에 대한 임베딩을 생성한다.
        response = embedding_request(text=result.title)
        # 파일 참조 정보: [논문 제목, 다운로드된 PDF 파일 경로, 제목 임베딩]
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response.data[0].embedding,
        ]

        # 파일에 참조 정보를 기록한다.
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list

In [8]:
# 검색 기능이 잘 작동하는지 테스트한다.
result_output = get_articles("rag")
result_output[0] # 첫 번째 검색 결과를 확인한다.

{'title': 'Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks',
 'summary': 'Retrieval-augmented Generation (RAG) has markedly enhanced the capabilities\nof Large Language Models (LLMs) in tackling knowledge-intensive tasks. The\nincreasing demands of application scenarios have driven the evolution of RAG,\nleading to the integration of advanced retrievers, LLMs and other complementary\ntechnologies, which in turn has amplified the intricacy of RAG systems.\nHowever, the rapid advancements are outpacing the foundational RAG paradigm,\nwith many methods struggling to be unified under the process of\n"retrieve-then-generate". In this context, this paper examines the limitations\nof the existing RAG paradigm and introduces the modular RAG framework. By\ndecomposing complex RAG systems into independent modules and specialized\noperators, it facilitates a highly reconfigurable framework. Modular RAG\ntranscends the traditional linear architecture, embracing a mor

In [9]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), # 코사인 유사도 계산 함수
    top_n: int = 100,
) -> list[str]:
    
    """가장 관련성이 높은 순으로 정렬된 문자열과 관련도 목록을 반환한다."""
    
    # 쿼리에 대한 임베딩을 생성한다.
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response.data[0].embedding

    # 데이터프레임의 각 항목과 쿼리 간의 관련도를 계산한다.
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    # 관련도(유사도)가 높은 순으로 정렬한다.
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]

In [12]:
def read_pdf(filepath):
    """PDF 파일 경로를 받아 PDF의 내용을 문자열로 반환한다."""
    # pdf reader 객체 생성
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


def create_chunks(text, n, tokenizer):
    """제공된 텍스트에서 n-크기의 청크(조각)를 연속적으로 반환한다."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # 0.5*n에서 1.5*n 토큰 범위 내에서 가장 가까운 문장 끝을 찾는다.
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # 토큰을 디코딩하여 마침표나 줄바꿈 문자가 있는지 확인한다.
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # 문장 끝을 찾지 못하면 n 토큰을 청크 크기로 사용한다.
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """이 함수는 입력 콘텐츠에 프롬프트를 적용한다. 이 경우 요약된 텍스트 청크를 반환한다."""
    prompt = template_prompt + content
    response = client.chat.completions.create(
        model=MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response.choices[0].message.content


def summarize_text(query):
    """
    이 함수는 다음을 수행한다:
    - 임베딩을 포함한 arxiv_library.csv 파일을 읽는다.
    - 사용자 쿼리와 가장 유사한 파일을 찾는다.
    - 파일에서 텍스트를 스크랩하여 청크로 나눈다.
    - 각 청크를 병렬로 요약한다.
    - 최종 요약을 수행하고 이를 사용자에게 반환한다.
    """
    # 재귀적 요약이 입력 논문에 어떻게 접근해야 하는지 지시하는 프롬프트
    summary_prompt = """학술 논문에서 발췌한 이 텍스트를 요약하고, 핵심 포인트와 근거를 추출해주세요.\n\n내용:"""

    # 라이브러리가 비어 있으면(아직 검색이 수행되지 않음), 먼저 검색을 수행하고 결과를 다운로드
    library_df = pd.read_csv(paper_dir_filepath, header=None).reset_index(drop=True)
    if len(library_df) == 0:
        print("아직 검색된 논문이 없습니다. 먼저 다운로드합니다.")
        get_articles(query)
        print("논문 다운로드 완료, 계속 진행합니다.")
        library_df = pd.read_csv(paper_dir_filepath, header=None).reset_index(drop=True)
    else:
        print(f"기존 논문 발견... 논문 수: {len(library_df)}")

    library_df.columns = ["title", "filepath", "embedding"]
    # 문자열 형태의 임베딩을 실제 리스트 객체로 변환한다.
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    # 쿼리와 가장 관련성이 높은 논문 파일 경로를 찾는다.
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)

    print("논문에서 텍스트를 청크로 나누는 중")
    pdf_text = read_pdf(strings[0])

    # 토크나이저 초기화
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # 문서를 1500 토큰 청크로 나눈다.
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("각 텍스트 청크 요약 중")

    # 요약을 병렬로 처리한다.
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(text_chunks)) as executor:
        futures = [executor.submit(extract_chunk, chunk, summary_prompt) for chunk in text_chunks]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # 최종 요약
    print("전체 요약으로 종합하는 중")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""학술 논문에서 추출한 이 핵심 포인트 모음에서 종합된 요약을 작성해주세요.
                        요약은 핵심 주장, 결론 및 증거를 강조하고 사용자의 질문에 답변해야 합니다.
                        사용자 질문: {query}
                        요약은 '핵심 주장', '증거', '결론'이라는 제목 아래 글머리 기호 목록으로 구조화되어야 합니다.
                        핵심 포인트:\n{results}\n요약:\n""",
            }
        ],
        temperature=0,
    )
    return response

In [13]:
# summarize_text 함수가 작동하는지 테스트한다.
chat_test_response = summarize_text("modular rag")

기존 논문 발견... 논문 수: 20
논문에서 텍스트를 청크로 나누는 중
각 텍스트 청크 요약 중


100%|██████████| 11/11 [00:17<00:00,  1.60s/it]


전체 요약으로 종합하는 중


In [14]:
# 테스트 응답의 내용을 마크다운 형식으로 표시한다.
display(Markdown(chat_test_response.choices[0].message.content))

## 핵심 주장

- **Modular RAG(모듈화된 Retrieval-Augmented Generation) 프레임워크는 기존 RAG 시스템의 한계를 극복하고, 복잡한 쿼리와 다양한 데이터 소스에 유연하고 확장 가능하게 대응할 수 있도록 설계된 혁신적 패러다임이다.**
- Modular RAG는 RAG 시스템을 독립적 모듈과 연산자(operator)로 분해하여, 라우팅, 스케줄링, 융합 등 다양한 워크플로우를 구성할 수 있게 하며, 기존 Naive RAG와 Advanced RAG를 포괄하는 상위 개념으로 발전시킨다.
- 이 프레임워크는 RAG 시스템의 설계, 관리, 유지보수를 용이하게 하고, 새로운 모듈과 연산자 개발을 촉진하여 RAG 기술의 이론적·실용적 발전을 견인한다.

## 증거

- **기존 RAG 한계**: Naive RAG는 단순 쿼리-문서 유사도 기반 검색에 의존해 복잡한 쿼리 처리에 취약하며, Advanced RAG도 검색 정확도 향상에 집중했으나 데이터 통합, 시스템 해석성, 유지보수 문제는 여전함. (예: “Shallow Understanding of Queries”, “Retrieval Redundancy and Noise” 문제)
- **Modular RAG 구조**: 3계층(모듈, 서브모듈, 연산자)로 RAG 시스템을 분해하고, 라우팅·스케줄링·융합 메커니즘을 포함해 비선형적이고 유연한 워크플로우를 구성함. 기존 RAG는 Modular RAG의 특수 사례임. (예: “Modular RAG system consists of multiple independent yet tightly coordinated modules”)
- **유연성 및 확장성**: 다양한 데이터 유형(비정형 텍스트, 테이블, 지식 그래프 등)과 복잡한 작업 시나리오에 맞게 모듈과 연산자를 조합 가능하며, 유지보수와 이해도가 향상됨. (예: “enhances the system’s flexibility and scalability”, “strengthens the system’s maintainability and comprehensibility”)
- **실제 적용 사례**: 인덱싱 모듈에서 문서 분할, 청크 최적화, 계층적 구조화, 메타데이터 부착 등을 통해 검색 정확도와 효율성을 높임. (예: “Chunk Optimization”, “Hierarchical Index”, “Metadata Attachment”)
- **다양한 RAG 흐름 패턴 지원**: 선형, 조건부, 분기, 루프 패턴 등 다양한 워크플로우를 지원하며, LLM 판단 및 제어, 지식 기반 스케줄링, 다중 분기 결과 융합 기법을 포함함.
- **미세조정과 검증 기법**: 검색기와 생성기의 미세조정, LLM-Critique, 외부 지식 기반 검증, 강화학습 등을 통해 성능과 신뢰성을 높임.
- **문헌 및 최신 연구 동향**: RAG와 LLM 결합, 계획 기반 검색, 자기 비판적 학습, 다중 에이전트 협업, 환각 문제 완화 등 다양한 최신 연구가 Modular RAG의 이론적·실용적 기반을 뒷받침함.

## 결론

- Modular RAG는 기존 RAG 시스템의 단점을 극복하고, 복잡하고 다양한 실제 응용 요구에 맞춰 RAG 시스템을 유연하고 확장 가능하게 설계할 수 있는 혁신적 프레임워크이다.
- 모듈화된 설계는 시스템의 유지보수성과 해석성을 크게 향상시키며, 새로운 모듈과 연산자 개발을 촉진하여 RAG 기술의 지속적 발전을 가능하게 한다.
- 특히, 인덱싱 단계의 최적화, 다양한 흐름 패턴 지원, LLM 기반 판단 및 제어, 미세조정과 검증 기법의 통합적 활용이 RAG 성능 향상의 핵심 요소임을 입증한다.
- Modular RAG는 RAG 기술의 이론적 발전과 실용적 적용을 위한 강력한 기반을 제공하며, 향후 복잡한 지식 집약적 NLP 작업과 대규모 시스템 구축에 필수적인 설계 패러다임으로 자리매김할 전망이다.

### 3\. 에이전트 구성 (Configure Agent)

In [15]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=MODEL):
    """ChatCompletion API에 요청을 보내는 헬퍼 함수"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # tools 매개변수가 최신 API에서는 권장되지만, 여기서는 functions를 사용한다.
            functions=functions,
        )
        return response
    except Exception as e:
        print("ChatCompletion 응답을 생성할 수 없습니다.")
        print(f"예외: {e}")
        return e

In [16]:
class Conversation:
    """대화 기록을 관리하는 클래스"""
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [17]:
# get_articles와 read_article_and_summarize 함수를 모델에게 알려주기 위한 명세(schema)
arxiv_functions = [
    {
        "name": "get_articles",
        "description": "사용자 질문에 답하기 위해 arXiv에서 학술 논문을 가져오려면 이 함수를 사용하세요.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "JSON 형식의 사용자 쿼리입니다. 응답은 요약되어야 하며 논문 URL 참조를 포함해야 합니다.",
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "read_article_and_summarize",
        "description": "사용자를 위해 전체 논문을 읽고 요약을 제공하려면 이 함수를 사용하세요. 대화에서 get_articles가 호출되기 전에는 절대로 이 함수를 호출해서는 안 됩니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "사용자 쿼리를 기반으로 한 논문의 일반 텍스트 설명입니다.",
                }
            },
            "required": ["query"],
        },
    }
]

In [18]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """이 함수는 함수를 추가하는 옵션으로 ChatCompletion API를 호출한다."""
    response = chat_completion_request(messages, functions)
    full_message = response.choices[0]
    # 모델이 함수 호출을 결정했는지 확인한다.
    if full_message.finish_reason == "function_call":
        print("함수 생성이 요청되어 함수를 호출합니다.")
        return call_arxiv_function(messages, full_message)
    else:
        print("함수가 필요하지 않아 사용자에게 응답합니다.")
        return response


def call_arxiv_function(messages, full_message):
    """
    모델이 필요하다고 판단할 때 함수 호출을 실행하는 함수.
    현재는 이 if문에 절을 추가하여 확장된다.
    """
    function_name = full_message.message.function_call.name

    if function_name == "get_articles":
        try:
            # 모델이 생성한 인자(arguments)를 파싱한다.
            parsed_output = json.loads(full_message.message.function_call.arguments)
            print("검색 결과를 가져오는 중")
            # 실제 함수를 실행한다.
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print("함수 실행 실패")
            print(f"오류 메시지: {e}")
        # 함수 실행 결과를 대화 기록에 추가한다.
        messages.append({
            "role": "function",
            "name": function_name,
            "content": str(results),
        })
        try:
            print("검색 결과를 요약하는 중")
            # 함수 실행 결과를 바탕으로 모델에게 다시 자연어 응답을 생성하도록 요청한다.
            response = chat_completion_request(messages)
            return response
        except Exception as e:
            print(type(e))
            raise Exception("함수 채팅 요청 실패")

    elif function_name == "read_article_and_summarize":
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print("논문을 찾고 읽는 중")
        summary = summarize_text(parsed_output["query"])
        return summary
    else:
        raise Exception("함수가 존재하지 않으므로 호출할 수 없습니다.")

### 4\. arXiv 대화 (arXiv Conversation)

In [19]:
# 시스템 메시지로 시작한다.
paper_system_message = """당신은 arXivGPT이며, 사용자 질문에 답변하기 위해 학술 논문을 가져오는 유용한 조수입니다.
당신은 고객이 질문에 대한 답을 찾기 위해 어떤 논문을 읽을지 결정할 수 있도록 논문을 명확하게 요약합니다.
사용자가 논문의 이름을 이해하고 클릭하여 접근할 수 있도록 항상 article_url과 제목을 제공합니다.
시작하세요!"""

# 대화 객체를 생성하고 시스템 메시지를 추가한다.
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

In [20]:
# 사용자 메시지를 추가한다.
paper_conversation.add_message("user", "안녕하세요, Modular RAG는 어떻게 작동하나요?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, 
    functions=arxiv_functions
)
assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

함수 생성이 요청되어 함수를 호출합니다.
검색 결과를 가져오는 중
검색 결과를 요약하는 중


Modular RAG에 대한 가장 관련성 높은 논문은 다음과 같습니다:

논문 제목: Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks  
URL: http://arxiv.org/abs/2407.21059v1  
요약:  
Retrieval-augmented Generation (RAG)은 대규모 언어 모델(LLM)의 지식집약적 과제 처리 능력을 크게 향상시켰습니다. 그러나 RAG 시스템이 점차 복잡해지고 빠르게 발전함에 따라 기존의 "검색 후 생성(retrieve-then-generate)"이라는 기본 RAG 패러다임으로 모든 방법을 통합하기 어려워졌습니다.  
이 논문에서는 기존 RAG의 한계를 지적하고, RAG 시스템을 독립 모듈과 특수화된 연산자로 분해하는 '모듈러 RAG(Modular RAG)' 프레임워크를 제안합니다. Modular RAG는 전통적인 선형 아키텍처를 넘어 라우팅, 스케줄링, 융합 메커니즘을 통합하는 재구성 가능하고 유연한 디자인을 특징으로 합니다.  
특히 Modular RAG는 다양한 RAG 패턴(선형, 조건부, 분기, 루프)을 분석하며, RAG 시스템 설계 및 배포에 혁신적인 기회를 제공합니다. 이 접근법은 RAG의 지속적인 발전과 실제 적용을 위한 이론적 토대와 실용적 로드맵을 제시합니다.

요약하자면, Modular RAG는 기존의 통합된 RAG 프로세스를 LEGO 블록처럼 분할 가능한 독립 모듈로 바꾸어, 각 모듈이 독립적으로 구성되고, 라우팅과 스케줄링, 데이터 융합 등의 복잡한 프로세스를 지원함으로써 RAG 시스템을 더 유연하고 재사용 가능하게 만드는 혁신적인 프레임워크입니다.

더 자세한 내용을 원하시면 위 URL에서 논문 전문을 참고해 보세요.

In [21]:
# 두 번째 도구를 사용하도록 유도하는 다른 사용자 메시지를 추가한다.
paper_conversation.add_message(
    "user",
    "RAG 파이프라인 논문을 읽고 요약해 주시겠어요?",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, 
    functions=arxiv_functions
)

display(Markdown(updated_response.choices[0].message.content))

함수 생성이 요청되어 함수를 호출합니다.
논문을 찾고 읽는 중
기존 논문 발견... 논문 수: 30
논문에서 텍스트를 청크로 나누는 중
각 텍스트 청크 요약 중


100%|██████████| 11/11 [00:18<00:00,  1.69s/it]


전체 요약으로 종합하는 중


## 핵심 주장

- **Modular RAG(모듈러 RAG)는 기존 RAG 시스템의 한계를 극복하고, LEGO 블록처럼 조립 가능한 독립적 모듈과 연산자(operator)로 RAG 시스템을 재구성하는 혁신적 프레임워크이다.**  
- 기존 Naive RAG는 단순 선형 구조로 복잡한 쿼리와 다양한 데이터 유형 처리에 취약하며, Advanced RAG는 검색 정확도는 개선했으나 시스템 복잡성과 유지보수 문제를 내포한다.  
- Modular RAG는 라우팅, 스케줄링, 융합 등 다양한 기능을 모듈화하여 유연하고 확장 가능한 RAG 아키텍처를 제공하며, 선형, 조건부, 분기, 반복 등 다양한 RAG 흐름 패턴을 포괄한다.  
- 모듈화된 설계는 시스템의 유지보수성, 해석 가능성, 제어성을 크게 향상시키고, 새로운 기술과 방법론과의 호환성 및 확장성을 보장한다.  
- Modular RAG는 RAG 기술의 이론적 토대와 실용적 발전을 위한 로드맵을 제시하며, 복잡한 실제 응용 시나리오에 적합한 프레임워크임을 입증한다.

## 증거

- **기존 RAG 한계**: Naive RAG는 쿼리-문서 유사도 기반 단순 검색에 의존해 복잡한 쿼리 처리에 취약하고, 중복 및 노이즈가 많아 LLM의 정확한 정보 추출을 방해함.  
- **Advanced RAG 한계**: 쿼리 재작성, 결과 재순위화 등으로 검색 정확도는 향상되었으나, 복잡한 워크플로우 관리와 이질적 데이터 통합에 어려움 존재.  
- **Modular RAG 구조**: RAG 시스템을 모듈, 서브모듈, 연산자 3단계로 분해하고, 이를 계산 그래프 형태로 표현하여 다양한 RAG 패턴(선형, 조건부, 분기, 반복)을 지원함.  
- **유연성 및 확장성**: 모듈과 연산자의 조합으로 다양한 데이터 소스와 작업 시나리오에 맞게 시스템을 맞춤화 가능하며, 독립적 모듈 설계로 유지보수 및 디버깅이 용이함.  
- **실제 적용 사례**: DR-RAG(2단계 검색 및 분류기 조합), PlanRAG(사전 계획 및 재계획 단계 도입), Multi-Head RAG(다중 주의층 활용) 등 다양한 사례에서 모듈러 RAG의 유연성과 강력함 입증.  
- **RAG 흐름 패턴**: 선형, 조건부, 분기, 루프(반복/재귀) 패턴을 체계적으로 분석하고, 각 패턴에 맞는 모듈 조합과 제어 메커니즘을 제시함.  
- **미세조정 및 검증 기법**: 검색기 및 생성기 미세조정, LLM 자체 평가, 외부 지식 기반 검증 등 다양한 성능 향상 및 신뢰성 확보 기법과의 통합 가능성.

## 결론

- Modular RAG는 RAG 시스템을 LEGO 블록처럼 독립적이고 재조합 가능한 모듈과 연산자로 분해하여, 복잡한 쿼리와 다양한 데이터 유형에 유연하게 대응할 수 있는 혁신적 프레임워크이다.  
- 이를 통해 기존 RAG의 한계를 극복하고, 유지보수성, 해석 가능성, 제어성, 확장성을 대폭 향상시킨다.  
- Modular RAG는 다양한 RAG 흐름 패턴(선형, 조건부, 분기, 반복)을 포괄하며, 실제 응용에서의 복잡한 워크플로우 관리와 최적화 문제를 해결한다.  
- 또한, 미세조정, 검증, 동적 라우팅 및 스케줄링 등 최신 기법과의 높은 호환성을 갖추어 RAG 기술의 이론적·실용적 발전을 위한 견고한 기반을 제공한다.  
- 따라서 Modular RAG는 RAG 시스템 설계 및 구현의 새로운 표준으로 자리매김할 가능성이 크며, 향후 연구 및 산업적 응용에서 핵심적인 역할을 할 것으로 기대된다.